In [2]:
import pandas as pd
import numpy as np

from bayes import *
from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import bernoulli, beta  
from scipy.stats import norm, sem
import scipy.stats as stats
import statsmodels.stats.api as sms

import pylab as plt

In [3]:
df = pd.read_csv('data/cookie_cats.csv')

df.columns = [i.lower().replace(' ','_') for i in df.columns]

In [4]:
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [5]:
# Eliminamos aquellos jugadores que no hayan realizado ninguna partida.

cerogame = df[df.sum_gamerounds == 0].index
df = df[~df.index.isin(cerogame)]
df.shape

(86195, 5)

In [6]:
df.version.value_counts()

version
gate_40    43432
gate_30    42763
Name: count, dtype: int64

In [7]:
imps_ctrl = 42763
convs_ctrl = len(df[(df.version == 'gate_30') & (df.retention_1 == True)])

imps_test = 43432
convs_test = (len(df[(df.version == 'gate_40') & (df.retention_1 == True)]))

In [8]:
CR_ctrl = convs_ctrl/imps_ctrl
CR_test = convs_test/imps_test

f'Tasas estancia: Control: {CR_ctrl}, Test: {CR_test}'

'Tasas estancia: Control: 0.46753034165049223, Test: 0.46217074967765703'

In [9]:
# calcular el tamaño del efecto. 46.75% a 48.75%
efecto = sms.proportion_effectsize(CR_ctrl, CR_ctrl + 0.2)

In [10]:
# tamaño de la muestra
n_requirido = sms.NormalIndPower().solve_power(efecto,
                                               power=0.99,
                                               alpha=0.01)

n_requirido

290.6407148839068

In [11]:
control = df[df.version == 'gate_30'].sample(n=291, random_state=42)

tratamiento = df[df.version == 'gate_40'].sample(n=291, random_state=42)

ab_test = pd.concat([control, tratamiento], axis=0)

ab_test.reset_index(drop=True, inplace=True)

ab_test.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,2741718,gate_30,15,True,False
1,1579627,gate_30,25,False,True
2,3558203,gate_30,5,False,False
3,50850,gate_30,34,True,True
4,6476239,gate_30,38,True,True


In [12]:
ab_test.version.value_counts()

version
gate_30    291
gate_40    291
Name: count, dtype: int64

In [13]:
tasas_conv = ab_test.groupby('version')['retention_1']

tasas_conv = tasas_conv.agg([np.mean,
                               lambda x: np.std(x, ddof=0),
                               lambda x: stats.sem(x, ddof=0)
                               ])


tasas_conv.columns = ['estancia_rate', 'std', 'sem']

tasas_conv.style.format('{:.3f}')

/var/folders/h7/xhgpgb3d0s55331krx9jbrv80000gn/T/ipykernel_2587/2202896903.py:3: FutureWarning: The provided callable <function mean at 0x105fb3420> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  tasas_conv = tasas_conv.agg([np.mean,


,estancia_rate,std,sem
version,,,
gate_30,0.491,0.500,0.029
gate_40,0.412,0.492,0.029


In [14]:
# selecciono grupos
control_res = ab_test[ab_test.version=='gate_30']['retention_1']

trat_res = ab_test[ab_test.version=='gate_40']['retention_1']

sum(control_res), sum(trat_res)

(143, 120)

In [17]:
impresiones = [control_res.shape[0], trat_res.shape[0]]  # entrar en al pagina 

conversiones = [sum(control_res), sum(trat_res)]     # comprar en la pagina



z_score, p_value = proportions_ztest(conversiones, nobs=impresiones)

(control_a, trata_a), (control_b, trata_b) = proportion_confint(conversiones, 
                                                                nobs=impresiones,
                                                                alpha = 0.05)

In [18]:
print(f'z-score: {z_score:.2f}')

print(f'p-valor: {p_value:.3f}')

print(f'intervalo conf 95% para grupo control: [{control_a:.3f}, {control_b:.3f}]')

print(f'intervalo conf 95% para grupo tratamiento: [{trata_a:.3f}, {trata_b:.3f}]')

z-score: 1.92
p-valor: 0.055
intervalo conf 95% para grupo control: [0.434, 0.549]
intervalo conf 95% para grupo tratamiento: [0.356, 0.469]


Dado que el p_valor es mayor que 0.01 (el grado de confianza establecido para la exploración) no podemos rechazar la hipótesis nula H0. Por tanto, el hecho de pasar la puerta de pago a la gate_40 no tiene un rendimiento significativamente diferente a tenerla en la gate_30.

También vemos que en ambos intervalos de confianza se encuentra incluido nuestro valor base de 48.75% de tasa de retención.
Esto contradice en cierta medida la conclusión obtenida en el p_valor.

